# Flat reference file generation example

In this notebook we illustrate how to generate a flat reference file from a set of pre-processed (L2) files using the `Flat` class.

In particular, we will focus on the `Flat.make_flat_from_files()` method.

In [ ]:
from wfi_reference_pipeline.reference_types.flat import flat

In [ ]:
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta

In [ ]:
import glob

import numpy as np

In [ ]:
# You can change this to whichever sensor between 1 and 18 (inclusive)
sca = 1
# Band number -- more notes below
band_no = 1

In this example we focus on pre-processed TVAC2 data. These data were taken during the CFA test.

The data are "internal flats" using the (s)RCS. What does this mean?

For these exposures the optical element is the "DARK" element, which contains a diffuser in the back.
This diffuser is illuminated by one (or 2) LEDs from the sRCS. The LEDs transmission curves greatly overlap with the tranmission curves of the
filters. So we approximate that these flats are approximately similar to those using the filter. This is not totally accurate as it will not capture
the spatial variations in throughput from the filters themselves, but it should give a good approximation for the wavelength dependence.

Additional notes: There is no band that overlaps with F213. For F184, the LED is bluer than the filter itself. F146 should be a combination of different flats.

In [ ]:
bands = ['F062', 'F087', 'F106', 'F129', 'F158', 'F184', 'F213', 'F146']

In [ ]:
# For now hardcoded. These data live in central store.
# These files can only be read with rad/roman_datamodels <= 0.21
flist = glob.glob(f'/PATH/TO/TVAC2/cal/NOM_OPS/OTP00614*/*WFI{sca:02d}_cal_v0_band{band_no}.asdf')

In [ ]:
len(flist)

In [ ]:
meta = MakeDevMeta(ref_type='FLAT')

In [ ]:
meta.meta_flat

In [ ]:
meta.meta_flat.instrument_detector = f'WFI{sca:02d}'
meta.meta_flat.ref_optical_element = bands[band_no - 1]
meta.meta_flat.pedigree = 'GROUND'

In [ ]:
# We use only a few of the exposures to create the flats
f = flat.Flat(meta_data=meta.meta_flat, file_list=flist[:10])

In [ ]:
test_flat, test_err = f.make_flat_from_files(calc_error=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(test_flat, origin='lower', vmin=0.6, vmax=1.4)
plt.colorbar()

In [ ]:
import asdf

In [ ]:
# This is what was created with the full dataset. Use as reference
test_data = asdf.open('/PATH/TO/flats_tvac2/roman_flat_WFI01_band1.asdf')

In [ ]:
plt.imshow(test_flat / test_data['roman']['data'] - 1, vmin=-0.03, vmax=0.03)

In [ ]:
plt.hist(test_flat.flatten() / test_data['roman']['data'].flatten() - 1, range=(-0.1, 0.1), bins=100, histtype='step')
plt.xlabel('Flat / Master flat - 1')
plt.ylabel('Number pixels / bin')

Everything looks within 2.5%

In [ ]:
# Visualize the relative uncertainty from bootstrap
plt.imshow(test_err / np.abs(test_flat), vmax=0.017, vmin=0.00)
plt.colorbar(label = 'Flat error / Flat')

In [ ]:
plt.hist(np.log10(test_err.flatten() / np.abs(test_flat.flatten())), bins=100, range=(-6, 1), histtype='step')
plt.xlabel(r'$\log_{10}(\mathrm{Flat error} / \mathrm{Flat})$')
plt.ylabel('Number of pixels / bin')